# Final Project: Paris High-End Restaurant Implementation

## Introduction/Business Problem

As one of the most visited cities in the world, with remarkable venues such as the Champs-Elysées (more than 30 million tourists each year), Paris is the epicenter of french tourism and cultural heritage. It is thus no doubt that opening a restaurant in Paris, with its potential for influx, is a choice that must be wisely considered and optimized.

But outside of the touristical places, Paris is also a city with an inner-urban population of more than 2 million people. In a COVID-19 impacted world, this population represents the major influx for restaurants, where foreign tourists are stuck in their home countries. Given the uncertainty of the virus, relying on this source of revenues seems the safer choice when opening a restaurant in the actual period.

We thus aim here at providing location tools and inputs for chefs that would wish to open an high-end restaurant in Paris in the COVID-19 context. We will base ourselves on venues exploration possibilities given by Foursquare and on data about revenues in Paris' neighbourhoods (as the objective is to be high-end and thus target the most revenue-intensive part of the population). Several criteria will be taken into account such as presence of other high-end restaurants and its intensity.

## The Data
For our problem we used three main sources of data:  
  
-Foursquare for the data about venues in Paris' neighbourhoods.  
-For neighbourhoods data about the city of Paris, the mayor website has an open data tab for various purposes. A Json file is provided for information about the neighbourhoods of Paris, called "Arrondissements" (available at https://opendata.paris.fr/explore/dataset/arrondissements/information/?disjunctive.c_ar&disjunctive.c_arinsee&disjunctive.l_ar&basemap=jawg.dark&location=12,48.89091,2.34678).  
-For the revenues in Paris neighbourhoods we will use an indicator which is the median of the disposable income per consumption unit. We used an official source from the website of the French statistical bureau (Insee) with data from 2007 (available at https://www.insee.fr/fr/statistiques/1405599?geo=DEP-75+COM-75101+COM-75102+COM-75103+COM-75104+COM-75105+COM-75106+COM-75107+COM-75108+COM-75109+COM-75110+COM-75111+COM-75112+COM-75113+COM-75114+COM-75115+COM-75116+COM-75117+COM-75118+COM-75119+COM-75120).  

## Data Preparation

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium==0.5
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                             \                            -failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='0_73_pypy|1_73_pypy|4_73_pypy|3_73_pypy|2_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']
  - rsa -> pyth

In [2]:
df=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS_0VhBGttQ9oZQPzasdgVFLaEbn5dnSmuaM5Al_IOiclfnQFzd4J1K2u3bjDEpCA/pub?output=csv')

In [3]:
df.drop(df.index[20], inplace=True)
df

,Identifiant sÃ©quentiel de lâ€™arrondissement,NumÃ©ro dâ€™arrondissement,NumÃ©ro dâ€™arrondissement INSEE,Nom de lâ€™arrondissement,Nom officiel de lâ€™arrondissement,N_SQ_CO,Surface,PÃ©rimÃ¨tre,Geometry X Y
0,750000001,1.0,75101.0,1er Ardt,Louvre,750001537.0,1.824613e+06,6054.936862,"48.8625627018,2.33644336205"
1,750000002,2.0,75102.0,2Ã¨me Ardt,Bourse,750001537.0,9.911537e+05,4554.104360,"48.8682792225,2.34280254689"
2,750000019,19.0,75119.0,19Ã¨me Ardt,Buttes-Chaumont,750001537.0,6.792651e+06,11253.182479,"48.8870759966,2.38482096015"
3,750000003,3.0,75103.0,3Ã¨me Ardt,Temple,750001537.0,1.170883e+06,4519.263648,"48.86287238,2.3600009859"
4,750000007,7.0,75107.0,7Ã¨me Ardt,Palais-Bourbon,750001537.0,4.090057e+06,8099.424883,"48.8561744288,2.31218769148"
5,750000005,5.0,75105.0,5Ã¨me Ardt,PanthÃ©on,750001537.0,2.539375e+06,6239.195396,"48.8444431505,2.35071460958"
6,750000008,8.0,75108.0,8Ã¨me Ardt,Ã‰lysÃ©e,750001537.0,3.880036e+06,7880.533268,"48.8727208374,2.3125540224"
7,750000017,17.0,75117.0,17Ã¨me Ardt,Batignolles-Monceau,750001537.0,5.668835e+06,10775.579516,"48.887326522,2.30677699057"
8,750000020,20.0,75120.0,20Ã¨me Ardt,MÃ©nilmontant,750001537.0,5.983446e+06,10704.940486,"48.8634605789,2.40118812928"
9,750000006,6.0,75106.0,6Ã¨me Ardt,Luxembourg,750001537.0,2.153096e+06,6483.686786,"48.8491303586,2.33289799905"


In [4]:
df.drop(['Identifiant sÃ©quentiel de lâ€™arrondissement', 'NumÃ©ro dâ€™arrondissement', 'Nom de lâ€™arrondissement', 'N_SQ_CO', 'Surface', 'PÃ©rimÃ¨tre'], axis=1, inplace=True)

In [5]:
df

,NumÃ©ro dâ€™arrondissement INSEE,Nom officiel de lâ€™arrondissement,Geometry X Y
0,75101.0,Louvre,"48.8625627018,2.33644336205"
1,75102.0,Bourse,"48.8682792225,2.34280254689"
2,75119.0,Buttes-Chaumont,"48.8870759966,2.38482096015"
3,75103.0,Temple,"48.86287238,2.3600009859"
4,75107.0,Palais-Bourbon,"48.8561744288,2.31218769148"
5,75105.0,PanthÃ©on,"48.8444431505,2.35071460958"
6,75108.0,Ã‰lysÃ©e,"48.8727208374,2.3125540224"
7,75117.0,Batignolles-Monceau,"48.887326522,2.30677699057"
8,75120.0,MÃ©nilmontant,"48.8634605789,2.40118812928"
9,75106.0,Luxembourg,"48.8491303586,2.33289799905"


In [6]:
df.dtypes

NumÃ©ro dâ€™arrondissement INSEE      float64
Nom officiel de lâ€™arrondissement     object
Geometry X Y                           object
dtype: object

In [7]:
df['NumÃ©ro dâ€™arrondissement INSEE']=df['NumÃ©ro dâ€™arrondissement INSEE'].astype(str)

In [8]:
df.rename(columns={"NumÃ©ro dâ€™arrondissement INSEE":"Code", "Nom officiel de lâ€™arrondissement":"Neighbourhood"}, inplace=True)
df

,Code,Neighbourhood,Geometry X Y
0,75101.0,Louvre,"48.8625627018,2.33644336205"
1,75102.0,Bourse,"48.8682792225,2.34280254689"
2,75119.0,Buttes-Chaumont,"48.8870759966,2.38482096015"
3,75103.0,Temple,"48.86287238,2.3600009859"
4,75107.0,Palais-Bourbon,"48.8561744288,2.31218769148"
5,75105.0,PanthÃ©on,"48.8444431505,2.35071460958"
6,75108.0,Ã‰lysÃ©e,"48.8727208374,2.3125540224"
7,75117.0,Batignolles-Monceau,"48.887326522,2.30677699057"
8,75120.0,MÃ©nilmontant,"48.8634605789,2.40118812928"
9,75106.0,Luxembourg,"48.8491303586,2.33289799905"


In [9]:
df['Neighbourhood']=df['Neighbourhood'].str.replace("Ã©","é")
df['Neighbourhood']=df['Neighbourhood'].str.replace("Ã´","ô")
df['Neighbourhood']=df['Neighbourhood'].str.replace("Ã‰","É")

In [10]:
df

,Code,Neighbourhood,Geometry X Y
0,75101.0,Louvre,"48.8625627018,2.33644336205"
1,75102.0,Bourse,"48.8682792225,2.34280254689"
2,75119.0,Buttes-Chaumont,"48.8870759966,2.38482096015"
3,75103.0,Temple,"48.86287238,2.3600009859"
4,75107.0,Palais-Bourbon,"48.8561744288,2.31218769148"
5,75105.0,Panthéon,"48.8444431505,2.35071460958"
6,75108.0,Élysée,"48.8727208374,2.3125540224"
7,75117.0,Batignolles-Monceau,"48.887326522,2.30677699057"
8,75120.0,Ménilmontant,"48.8634605789,2.40118812928"
9,75106.0,Luxembourg,"48.8491303586,2.33289799905"


In [11]:
df['Latitude'], df['Longitude'] = df['Geometry X Y'].str.split(',', 1).str

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  if __name__ == '__main__':


In [12]:
df.head()

,Code,Neighbourhood,Geometry X Y,Latitude,Longitude
0,75101.0,Louvre,"48.8625627018,2.33644336205",48.8625627018,2.33644336205
1,75102.0,Bourse,"48.8682792225,2.34280254689",48.8682792225,2.34280254689
2,75119.0,Buttes-Chaumont,"48.8870759966,2.38482096015",48.8870759966,2.38482096015
3,75103.0,Temple,"48.86287238,2.3600009859",48.86287238,2.3600009859
4,75107.0,Palais-Bourbon,"48.8561744288,2.31218769148",48.8561744288,2.31218769148


In [13]:
df['Code'] = df['Code'].str.replace(r'.0$', '')
df.head()

,Code,Neighbourhood,Geometry X Y,Latitude,Longitude
0,75101,Louvre,"48.8625627018,2.33644336205",48.8625627018,2.33644336205
1,75102,Bourse,"48.8682792225,2.34280254689",48.8682792225,2.34280254689
2,75119,Buttes-Chaumont,"48.8870759966,2.38482096015",48.8870759966,2.38482096015
3,75103,Temple,"48.86287238,2.3600009859",48.86287238,2.3600009859
4,75107,Palais-Bourbon,"48.8561744288,2.31218769148",48.8561744288,2.31218769148


In [14]:
df.drop('Geometry X Y', axis=1, inplace=True)

In [15]:
df.head()

,Code,Neighbourhood,Latitude,Longitude
0,75101,Louvre,48.8625627018,2.33644336205
1,75102,Bourse,48.8682792225,2.34280254689
2,75119,Buttes-Chaumont,48.8870759966,2.38482096015
3,75103,Temple,48.86287238,2.3600009859
4,75107,Palais-Bourbon,48.8561744288,2.31218769148


In [16]:
df.shape

(20, 4)

In [17]:
revenues=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRKPXsvo3kK8Ahhm7ZPDpNTP8ny8qek4Vi4_0gkAph_e2640u3-k2oBA9Wbi7wFCA/pub?output=csv')
revenues.head()

,Revenus,75101,75102,75103,75104,75105,75106,75107,75108,75109,75110,75111,75112,75113,75114,75115,75116,75117,75118,75119,75120
0,Médiane du revenu disponible par unité de cons...,32890,31470,32310,31200,33350,39330,43270,41270,33980,26370,27460,27700,24110,27680,30880,38900,30800,22330,20320,21560


In [18]:
revenues_paris=revenues.transpose()

In [19]:
revenues_paris

,0
Revenus,Médiane du revenu disponible par unité de cons...
75101,32890
75102,31470
75103,32310
75104,31200
75105,33350
75106,39330
75107,43270
75108,41270
75109,33980


In [20]:
revenues_paris=revenues_paris.reset_index()
revenues_paris

,index,0
0,Revenus,Médiane du revenu disponible par unité de cons...
1,75101,32890
2,75102,31470
3,75103,32310
4,75104,31200
5,75105,33350
6,75106,39330
7,75107,43270
8,75108,41270
9,75109,33980


In [21]:
header_row=0
revenues_paris.columns=revenues_paris.iloc[header_row]
revenues_paris

,Revenus,"Médiane du revenu disponible par unité de consommation en 2017, en euros"
0,Revenus,Médiane du revenu disponible par unité de cons...
1,75101,32890
2,75102,31470
3,75103,32310
4,75104,31200
5,75105,33350
6,75106,39330
7,75107,43270
8,75108,41270
9,75109,33980


In [22]:
revenues_paris=revenues_paris.drop(header_row)
revenues_paris = revenues_paris.reset_index(drop=True)

In [23]:
revenues_paris.head()

,Revenus,"Médiane du revenu disponible par unité de consommation en 2017, en euros"
0,75101,32890
1,75102,31470
2,75103,32310
3,75104,31200
4,75105,33350


In [24]:
revenues_paris.rename(columns={"Revenus":"Code", "Médiane du revenu disponible par unité de consommation en 2017, en euros":"Revenues_per_con_unit"}, inplace=True)
revenues_paris.head()

,Code,Revenues_per_con_unit
0,75101,32890
1,75102,31470
2,75103,32310
3,75104,31200
4,75105,33350


In [25]:
revenues_paris.dtypes

0
Code                     object
Revenues_per_con_unit    object
dtype: object

In [26]:
df['Longitude'] = df['Longitude'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)

In [27]:
left_join=pd.merge(df, revenues_paris, on='Code', how='left')
left_join.head()

,Code,Neighbourhood,Latitude,Longitude,Revenues_per_con_unit
0,75101,Louvre,48.862563,2.336443,32890
1,75102,Bourse,48.868279,2.342803,31470
2,75119,Buttes-Chaumont,48.887076,2.384821,20320
3,75103,Temple,48.862872,2.360001,32310
4,75107,Palais-Bourbon,48.856174,2.312188,43270


In [28]:
left_join.isnull().sum()

Code                     0
Neighbourhood            0
Latitude                 0
Longitude                0
Revenues_per_con_unit    0
dtype: int64

## Data Exploration and Analysis

In [29]:
address = 'Paris, FR'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


In [30]:
map_paris = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(left_join['Latitude'], left_join['Longitude'], left_join['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

### Foursquare API requests

In [31]:
CLIENT_ID = 'LTMSXTZLYYU5AYPNKNSO4YGTHLBUL1ASKTLVVIXOJSNNB0UM' # your Foursquare ID
CLIENT_SECRET = 'RFX232Y04H2QXBYZRAZGDPYLDXTIULSJLFLS4RREEGSXP4P3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LTMSXTZLYYU5AYPNKNSO4YGTHLBUL1ASKTLVVIXOJSNNB0UM
CLIENT_SECRET:RFX232Y04H2QXBYZRAZGDPYLDXTIULSJLFLS4RREEGSXP4P3


In [32]:
radius = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
paris_venues = getNearbyVenues(names=left_join['Neighbourhood'],
                                   latitudes=left_join['Latitude'],
                                   longitudes=left_join['Longitude']
                                  )

Louvre
Bourse
Buttes-Chaumont
Temple
Palais-Bourbon
Panthéon
Élysée
Batignolles-Monceau
Ménilmontant
Luxembourg
Popincourt
Gobelins
Opéra
Buttes-Montmartre
Hôtel-de-Ville
Observatoire
Passy
Entrepôt
Vaugirard
Reuilly


In [34]:
paris_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Louvre,48.862563,2.336443,Musée du Louvre,48.860847,2.336440,Art Museum
1,Louvre,48.862563,2.336443,Palais Royal,48.863236,2.337127,Historic Site
2,Louvre,48.862563,2.336443,Comédie-Française,48.863088,2.336612,Theater
3,Louvre,48.862563,2.336443,Place du Palais Royal,48.862523,2.336688,Plaza
4,Louvre,48.862563,2.336443,Cour Napoléon,48.861172,2.335088,Plaza


In [35]:
paris_venues.shape

(1273, 7)

In [36]:
paris_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Batignolles-Monceau,59,59,59,59,59,59
Bourse,100,100,100,100,100,100
Buttes-Chaumont,44,44,44,44,44,44
Buttes-Montmartre,44,44,44,44,44,44
Entrepôt,100,100,100,100,100,100
Gobelins,62,62,62,62,62,62
Hôtel-de-Ville,100,100,100,100,100,100
Louvre,82,82,82,82,82,82
Luxembourg,49,49,49,49,49,49


In [37]:
print('There are {} uniques categories.'.format(len(paris_venues['Venue Category'].unique())))

There are 204 uniques categories.


In [38]:
#We now analyze each neighbourhood. First one hot encoding then we group rows by neighbourhoods.
# one hot encoding
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Neighbourhood'] = paris_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Cambodian Restaurant,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soba Restaurant,South American Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,Louvre,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Louvre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Louvre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [39]:
paris_grouped = paris_onehot.groupby('Neighbourhood').mean().reset_index()
paris_grouped

,Neighbourhood,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,Baby Store,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Cambodian Restaurant,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soba Restaurant,South American Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,Batignolles-Monceau,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050847,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050847,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,0.033898,0.000000,0.000000,0.00000,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.016949,0.00,0.00,0.000000,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.016949,0.000000,0.000000,0.000000,0.169492,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000,0.00,0.000000,0.169492,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.067797,0.050847,0.000000,0.00,0.000000,0.033898,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.016949,0.0

In [40]:
#top 10 venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
paris_sorted = pd.DataFrame(columns=columns)
paris_sorted['Neighbourhood'] = paris_grouped['Neighbourhood']

for ind in np.arange(paris_grouped.shape[0]):
    paris_sorted.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

paris_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Batignolles-Monceau,Hotel,French Restaurant,Italian Restaurant,Japanese Restaurant,Bakery,Restaurant,Bistro,Plaza,Café,Korean Restaurant
1,Bourse,French Restaurant,Cocktail Bar,Wine Bar,Hotel,Bakery,Italian Restaurant,Bistro,Creperie,Cheese Shop,Coffee Shop
2,Buttes-Chaumont,French Restaurant,Bar,Café,Hotel,Bistro,Beer Bar,Brewery,Seafood Restaurant,Supermarket,Pizza Place
3,Buttes-Montmartre,Bar,French Restaurant,Hotel,Café,Coffee Shop,Convenience Store,Restaurant,Supermarket,Beer Store,Gastropub
4,Entrepôt,French Restaurant,Bistro,Hotel,Indian Restaurant,Coffee Shop,Café,Pizza Place,Japanese Restaurant,Korean Restaurant,Bar


### Clustering

In [42]:
#Clustering
kclusters = 5

paris_grouped_clustering = paris_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 1, 2, 1, 1, 1, 1], dtype=int32)

In [43]:
#DF with top 10 venues
paris_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

paris_merged = left_join

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
paris_merged = paris_merged.join(paris_sorted.set_index('Neighbourhood'), on='Neighbourhood')

paris_merged.head()

,Code,Neighbourhood,Latitude,Longitude,Revenues_per_con_unit,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75101,Louvre,48.862563,2.336443,32890,1,French Restaurant,Plaza,Hotel,Japanese Restaurant,Coffee Shop,Art Museum,Italian Restaurant,Wine Bar,Udon Restaurant,Garden
1,75102,Bourse,48.868279,2.342803,31470,1,French Restaurant,Cocktail Bar,Wine Bar,Hotel,Bakery,Italian Restaurant,Bistro,Creperie,Cheese Shop,Coffee Shop
2,75119,Buttes-Chaumont,48.887076,2.384821,20320,1,French Restaurant,Bar,Café,Hotel,Bistro,Beer Bar,Brewery,Seafood Restaurant,Supermarket,Pizza Place
3,75103,Temple,48.862872,2.360001,32310,1,French Restaurant,Italian Restaurant,Art Gallery,Japanese Restaurant,Coffee Shop,Gourmet Shop,Sandwich Place,Bakery,Cocktail Bar,Wine Bar
4,75107,Palais-Bourbon,48.856174,2.312188,43270,0,Hotel,French Restaurant,Italian Restaurant,Plaza,Café,History Museum,Cocktail Bar,Historic Site,Japanese Restaurant,Gourmet Shop


### Folium map 

In [83]:
# Map of clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['Neighbourhood'], paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Clusters description

In [84]:
#Cluster 1
paris_merged.loc[paris_merged['Cluster Labels'] == 0, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Palais-Bourbon,0,Hotel,French Restaurant,Italian Restaurant,Plaza,Café,History Museum,Cocktail Bar,Historic Site,Japanese Restaurant,Gourmet Shop
6,Élysée,0,French Restaurant,Hotel,Bakery,Spa,Department Store,Cocktail Bar,Resort,Corsican Restaurant,Plaza,Italian Restaurant
7,Batignolles-Monceau,0,Hotel,French Restaurant,Italian Restaurant,Japanese Restaurant,Bakery,Restaurant,Bistro,Plaza,Café,Korean Restaurant
15,Observatoire,0,French Restaurant,Hotel,Bistro,Italian Restaurant,Bakery,Brasserie,Fast Food Restaurant,Supermarket,Sushi Restaurant,Tea Room


In [85]:
#Cluster 2
paris_merged.loc[paris_merged['Cluster Labels'] == 1, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Louvre,1,French Restaurant,Plaza,Hotel,Japanese Restaurant,Coffee Shop,Art Museum,Italian Restaurant,Wine Bar,Udon Restaurant,Garden
1,Bourse,1,French Restaurant,Cocktail Bar,Wine Bar,Hotel,Bakery,Italian Restaurant,Bistro,Creperie,Cheese Shop,Coffee Shop
2,Buttes-Chaumont,1,French Restaurant,Bar,Café,Hotel,Bistro,Beer Bar,Brewery,Seafood Restaurant,Supermarket,Pizza Place
3,Temple,1,French Restaurant,Italian Restaurant,Art Gallery,Japanese Restaurant,Coffee Shop,Gourmet Shop,Sandwich Place,Bakery,Cocktail Bar,Wine Bar
5,Panthéon,1,French Restaurant,Italian Restaurant,Science Museum,Bakery,Hotel,Plaza,Pub,Café,Coffee Shop,Bistro
8,Ménilmontant,1,Bakery,Japanese Restaurant,Plaza,French Restaurant,Bar,Italian Restaurant,Bistro,Café,Diner,Cheese Shop
9,Luxembourg,1,Pastry Shop,Plaza,Bakery,French Restaurant,Fountain,Chocolate Shop,Hotel,Pharmacy,Deli / Bodega,Lebanese Restaurant
10,Popincourt,1,French Restaurant,Restaurant,Café,Supermarket,Pastry Shop,Italian Restaurant,Bakery,Bar,Wine Bar,Cocktail Bar
12,Opéra,1,French Restaurant,Hotel,Bakery,Cocktail Bar,Bistro,Japanese Restaurant,Wine Bar,Lounge,Restaurant,Italian Restaurant
13,Buttes-Montmartre,1,Bar,French Restaurant,Hotel,Café,Coffee Shop,Convenience Store,Restaurant,Supermarket,Beer Store,Gastropub


In [86]:
#Cluster 3
paris_merged.loc[paris_merged['Cluster Labels'] == 2, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Gobelins,2,Vietnamese Restaurant,Asian Restaurant,Chinese Restaurant,Thai Restaurant,French Restaurant,Bakery,Juice Bar,Italian Restaurant,Cambodian Restaurant,Sandwich Place


In [87]:
#Cluster 4
paris_merged.loc[paris_merged['Cluster Labels'] == 3, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Reuilly,3,Zoo Exhibit,Supermarket,Monument / Landmark,Zoo,Antique Shop,African Restaurant,Fountain,Food & Drink Shop,Food,Flower Shop


In [88]:
#Cluster 5
paris_merged.loc[paris_merged['Cluster Labels'] == 4, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Passy,4,Plaza,Lake,French Restaurant,Pool,Bus Station,Art Museum,Boat or Ferry,Bus Stop,Park,Ethiopian Restaurant


### Choropleth Map with revenues

In [89]:
# The code was removed by Watson Studio for sharing.

In [90]:
from ibm_botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name='s3', 
    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_1['ENDPOINT'])

In [91]:
cos.download_file(Bucket=credentials_1['BUCKET'],Key='arrondissements(2).json',Filename='arrondissements(2).json')

In [92]:
#Choropleth map of revenues areas
paris_chloro= 'arrondissements(2).json'
map_clusters.choropleth(geo_data=paris_chloro, data=left_join, columns=['Neighbourhood', 'Revenues_per_con_unit'], key_on='feature.properties.l_aroff', fill_color='BuPu', fill_opacity = 0.7, line_opacity=0.2, legend_name='Disposible income median per unit of consumption, in euros')
map_clusters